## IRA

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import missingno as msno
from pandas.plotting import parallel_coordinates
%matplotlib inline 

In [3]:
df_matriculas = pd.DataFrame()

# importando notas de 2010 a 2017
for i in range(10,18):
    for j in range(1,3):
        print('20'+str(i)+str(j))
        df_matricula = pd.read_csv('data_science/matriculas_new/matricula-componente-20'+str(i)+str(j)+'.csv',sep=';')
        #concatena as tabelas de 2010 a 2017
        df_matriculas = pd.concat([df_matriculas, df_matricula])

# obtendo o id do curso de ciências e tecnologia
df_cursos = pd.read_csv('data_science/cursos-graduacao.csv', sep=';')
id_cet = df_cursos[df_cursos.nome == 'CIÊNCIAS E TECNOLOGIA'].id_curso
id_cet

20101
20102
20111
20112
20121
20122
20131
20132
20141
20142
20151
20152
20161
20162
20171
20172


17    10320810
Name: id_curso, dtype: int64

In [4]:
# filtra tabela pelo id_curso
df_matriculasCeT = df_matriculas[df_matriculas.id_curso == float(id_cet)]

# columns = ['descricao', 'discente', 'faltas_unidade', 'id_curso', 'id_turma', 'media_final', 'nota', 'numero_total_faltas', 'reposicao', 'unidade']
# df_matriculas = df_matriculas[columns]

In [5]:
df_matriculas[df_matriculas.discente=='72eca9ca3b69e98cb6b5512b3d834913']

,Unnamed: 10,descricao,discente,faltas_unidade,id_curso,id_turma,media_final,nota,numero_total_faltas,reposicao,unidade
41300,NaN,APROVADO,72eca9ca3b69e98cb6b5512b3d834913,0.0,10320810.0,1208093,8.5,9.6,0.0,f,1.0
41301,NaN,APROVADO,72eca9ca3b69e98cb6b5512b3d834913,0.0,10320810.0,1208093,8.5,7.5,0.0,f,2.0
44246,NaN,APROVADO,72eca9ca3b69e98cb6b5512b3d834913,0.0,10320810.0,1208117,5.0,5.5,0.0,f,1.0
44247,NaN,APROVADO,72eca9ca3b69e98cb6b5512b3d834913,0.0,10320810.0,1208117,5.0,4.3,0.0,f,2.0
47903,NaN,APROVADO,72eca9ca3b69e98cb6b5512b3d834913,0.0,10320810.0,1208086,7.3,6.5,0.0,f,1.0
47904,NaN,APROVADO,72eca9ca3b69e98cb6b5512b3d834913,0.0,10320810.0,1208086,7.3,7.9,0.0,f,2.0
78899,NaN,APROVADO,72eca9ca3b69e98cb6b5512b3d834913,0.0,10320810.0,1208133,8.3,7.2,0.0,f,1.0
78900,NaN,APROVADO,72eca9ca3b69e98cb6b5512b3d834913,0.0,10320810.0,1208133,8.3,8.7,0.0,f,2.0
78901,NaN,APROVADO,72eca9ca3b69e98cb6b5512b3d834913,0.0,10320810.0,1208133,8.3,8.6,0.0,f,3.0
80041,NaN,APROVADO,72eca9ca3b69e98cb6b5512b3d834913,0.0,10320810.0,1208086,7.3,6.5,0.0,f,1.0


In [6]:
df_matriculasCeT.columns

Index(['Unnamed: 10', 'descricao', 'discente', 'faltas_unidade', 'id_curso',
       'id_turma', 'media_final', 'nota', 'numero_total_faltas', 'reposicao',
       'unidade'],
      dtype='object')

In [8]:
df_turmas = pd.read_csv('data_science/turmas-2010-2017-ok.csv')
df_turmas.columns

/home/andressa/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['Unnamed: 0', 'id_componente', 'codigo_turma', 'id_docente_interno',
       'id_docente_externo', 'observacao', 'id_componente_curricular',
       'nivel_ensino', 'campus_turma', 'local', 'ano', 'periodo',
       'data_inicio', 'data_fim', 'descricao_horario', 'total_solicitacoes',
       'capacidade_aluno', 'tipo', 'distancia', 'data_consolidacao',
       'agrupadora', 'id_turma_agrupadora', 'qtd_aulas_lancadas',
       'situacao_turma', 'convenio', 'modalidade_participantes'],
      dtype='object')

In [9]:
df_componentes = pd.read_csv('data_science/componentes-curriculares-presenciais-ok.csv')
df_componentes.columns

Index(['Unnamed: 0', 'id_componente', 'tipo_componente', 'codigo', 'nivel',
       'nome', 'unidade_responsavel', 'ch_teorico', 'ch_pratico', 'ch_estagio',
       'ch_total', 'ch_dedicada_docente', 'ch_ead', 'cr_max_ead',
       'permite_horario_flexivel', 'qtd_unidades', 'procedimentos_avaliacao',
       'equivalencia', 'pre_requisito', 'co_requisito', 'ementa',
       'bibliografia', 'objetivos', 'conteudo', 'competencias_habilidades',
       'referencias', 'ano_programa', 'periodo_programa', 'modalidade',
       'curso_componente'],
      dtype='object')

In [10]:
df_componentes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34951 entries, 0 to 34950
Data columns (total 30 columns):
Unnamed: 0                  34951 non-null int64
id_componente               34951 non-null int64
tipo_componente             34951 non-null object
codigo                      34951 non-null object
nivel                       34951 non-null object
nome                        34950 non-null object
unidade_responsavel         34951 non-null object
ch_teorico                  34951 non-null int64
ch_pratico                  34951 non-null int64
ch_estagio                  34951 non-null int64
ch_total                    34951 non-null int64
ch_dedicada_docente         34951 non-null int64
ch_ead                      34951 non-null int64
cr_max_ead                  34951 non-null int64
permite_horario_flexivel    34951 non-null object
qtd_unidades                28194 non-null float64
procedimentos_avaliacao     4448 non-null object
equivalencia                10332 non-null object


In [12]:
df_discentes = pd.read_csv('../secret/matriculas-2013-2017.csv', sep=';')

In [13]:
colunas_mat = ['discente', 'id_turma', 'media_final']
df_matriculasCeT = df_matriculasCeT[colunas_mat].dropna().drop_duplicates()

In [14]:
df_matriculasCeT.head(2)

,discente,id_turma,media_final
85637,4746f89573bcb024736f649807753654,57549019,8.6
85640,6ac3d868cc23be038b7b6e70afbd372b,57549019,7.6


In [20]:
print(df_turmas.info())
print(df_componentes.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204609 entries, 0 to 204608
Data columns (total 26 columns):
Unnamed: 0                  204609 non-null int64
id_componente               204609 non-null int64
codigo_turma                204578 non-null object
id_docente_interno          195250 non-null float64
id_docente_externo          9359 non-null float64
observacao                  7404 non-null object
id_componente_curricular    204609 non-null int64
nivel_ensino                204609 non-null object
campus_turma                141577 non-null object
local                       202540 non-null object
ano                         204609 non-null int64
periodo                     204609 non-null int64
data_inicio                 204609 non-null object
data_fim                    204609 non-null object
descricao_horario           180311 non-null object
total_solicitacoes          170050 non-null float64
capacidade_aluno            187177 non-null float64
tipo                       

In [19]:
df_turmasNome = pd.merge(df_turmas, df_componentes, on='id_componente')
df_turmasNome.head(2)

,Unnamed: 0_x,id_componente,codigo_turma,id_docente_interno,id_docente_externo,observacao,id_componente_curricular,nivel_ensino,campus_turma,local,...,ementa,bibliografia,objetivos,conteudo,competencias_habilidades,referencias,ano_programa,periodo_programa,modalidade,curso_componente


In [17]:
df_turmasNome.columns

Index(['Unnamed: 0_x', 'id_componente_x', 'codigo_turma', 'id_docente_interno',
       'id_docente_externo', 'observacao', 'id_componente_curricular',
       'nivel_ensino', 'campus_turma', 'local', 'ano', 'periodo',
       'data_inicio', 'data_fim', 'descricao_horario', 'total_solicitacoes',
       'capacidade_aluno', 'tipo', 'distancia', 'data_consolidacao',
       'agrupadora', 'id_turma_agrupadora', 'qtd_aulas_lancadas',
       'situacao_turma', 'convenio', 'modalidade_participantes',
       'Unnamed: 0_y', 'id_componente_y', 'tipo_componente', 'codigo', 'nivel',
       'nome', 'unidade_responsavel', 'ch_teorico', 'ch_pratico', 'ch_estagio',
       'ch_total', 'ch_dedicada_docente', 'ch_ead', 'cr_max_ead',
       'permite_horario_flexivel', 'qtd_unidades', 'procedimentos_avaliacao',
       'equivalencia', 'pre_requisito', 'co_requisito', 'ementa',
       'bibliografia', 'objetivos', 'conteudo', 'competencias_habilidades',
       'referencias', 'ano_programa', 'periodo_programa', 'm

In [16]:
colunas_turma = ['id_turma', 'nome', 'ch_total']
df_turmasNome = df_turmasNome[colunas_turma]
df_turmasNome.head(2)

KeyError: "['id_turma'] not in index"

In [64]:
df_ira = pd.merge(df_matriculasCeT, df_turmasNome, on='id_turma').drop_duplicates()
df_ira.head(2)

,discente,id_turma,media_final,nome,ch_total
0,d4245b974fc45e2ef383af9c74d4aa89,57548379,9.0,LINGUA INGLESA IX,60
1,747fc6ff3db5cc48b86765c3372219e9,57549019,8.6,TÓPICOS ESPECIAIS EM COMPUTAÇÃO XIV,60


In [67]:
df_ira['notaXch'] = df_ira['media_final']*df_ira['ch_total']

In [68]:
df_sumNotaCh = df_ira.groupby('discente').notaXch.sum().reset_index()
display(df_sumNotaCh.head(2))

df_sumCh = df_ira.groupby('discente').ch_total.sum().reset_index()
display(df_sumCh.head(2))

,discente,notaXch
0,0004f23716a0a9743b48969a5db7970f,10243.5
1,0005a138027eba573d7f9894f5721188,17634.0


,discente,ch_total
0,0004f23716a0a9743b48969a5db7970f,1995
1,0005a138027eba573d7f9894f5721188,3030


In [69]:
df_calIra = pd.merge(df_sumNotaCh, df_sumCh, on='discente')
df_calIra['ira'] = df_calIra['notaXch']/df_calIra['ch_total']

df_calIra.head(2)

,discente,notaXch,ch_total,ira
0,0004f23716a0a9743b48969a5db7970f,10243.5,1995,5.134586
1,0005a138027eba573d7f9894f5721188,17634.0,3030,5.819802


In [71]:
df_calIra[df_calIra['discente'] == '8032c974c90eae91a881657fc16092f2']

,discente,notaXch,ch_total,ira
4876,8032c974c90eae91a881657fc16092f2,20803.0,2410,8.63195
